In [ ]:
import numpy as np
from numpy import linalg
from numba import jit
import seaborn as sns
import matplotlib as mpl
from matplotlib import cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.transforms as mtransforms
from matplotlib.gridspec import SubplotSpec
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
from cycler import cycler

from scipy.constants import pi
from scipy.interpolate import RectBivariateSpline, griddata

import os
import ast
import pandas as pd

mpl.pyplot.rcdefaults()
plt.rcParams['savefig.facecolor'] = "white"
mpl.rcParams['figure.dpi'] = 300
mpl.rcParams['axes.linewidth'] = 1.2
mpl.rcParams['text.usetex'] = True
mpl.rcParams['mathtext.fontset'] = 'cm'
mpl.rcParams['font.family'] = 'CMU serif'

res = 300 #dpi

def set_pandas_display_options() -> None:
    """Set pandas display options."""
    # Ref: https://stackoverflow.com/a/52432757/
    display = pd.options.display

    display.max_columns = None
    display.max_rows = None
    display.max_colwidth = None
    display.width = None
    display.chop_threshold = None
    display.precision = 14  # set as needed

set_pandas_display_options()

# Import data

## Read files for parameters

In [ ]:
files = [x[0] for x in os.walk('./data/')][1::]
df = pd.DataFrame()
for file in files:
    data = pd.read_json(file+'/params.json',orient='index').transpose()
    data["key"] = file[7:]
    
    if "gap_pos" not in data:
        data["gap_pos"]= "bottom"
    if "block" not in data:
        data["block"]= "1"
    
    df = pd.concat([df,data])
    
display(df.set_index('key').sort_index(ascending=False))

## Select data

In [ ]:
# key = '1735335126'   # shift 0 bottom gap 1 upper block
key = '1735512504'    # shift 0 middle gap 1 upper block

## Import file and extract data

In [ ]:
outdir = './data/' + key

# parameter
tex, size = df.set_index('key').at[key,'texture'], df.set_index('key').at[key,'system_sizes'],
t, m = df.set_index('key').at[key,'t'], df.set_index('key').at[key,'m'],
shift, mag = df.set_index('key').at[key,'shift'], df.set_index('key').at[key,'mag']
thetas, fluxes = df.set_index('key').at[key,'q'], df.set_index('key').at[key,'flux']
t1t2, u1u2t1t2 = df.set_index('key').at[key,'tau1tau2'], df.set_index('key').at[key,'tau1tau2u1u2']
block, gap_pos = df.set_index('key').at[key,'block'], df.set_index('key').at[key,'gap_pos']

if block == 0:
    gap = 'lower block, gap ' + str(t1t2)
elif block ==1:
    gap = 'upper block, gap ' + str(t1t2)

n_theta = len(thetas)
n_flux  = len(fluxes)
    
fermi      = np.load(outdir+"/fermi.npy").real.reshape((n_theta,n_flux))
pre_fermi  = np.load(outdir+"/pre_fermi.npy").real.reshape((n_theta,n_flux))
post_fermi = np.load(outdir+"/post_fermi.npy").real.reshape((n_theta,n_flux))

ch_t1t2   = np.load(outdir+"/ch_tau1tau2.npy").real.reshape((n_theta,n_flux))
morbTotal = np.load(outdir+"/morb.npy").real.reshape((n_theta,n_flux))
morbLC    = np.load(outdir+"/morbLC.npy").real.reshape((n_theta,n_flux))
morbIC    = np.load(outdir+"/morbIC.npy").real.reshape((n_theta,n_flux))

pre_processed = False

In [ ]:
# for i in range(len(thetas)*len(fluxes)):
#     j = i % len(fluxes) # flux
#     k = i //len(fluxes) # theta
    
#     ids = 1 + t1t2 * fluxes[j] - u1u2t1t2 * thetas[k]**2
#     print(thetas[k],fluxes[j],int(ids*size**2),pre_fermi[k,j],fermi[k,j],post_fermi[k,j],ch_t1t2[k,j])

## Preprocessing

In [ ]:
if not pre_processed:
    # adjust Fermi energy
    if gap_pos == "middle":
        # morbTotal = morbTotal + 2 * (post_fermi+fermi)/2 * ch_tau1tau2.real/(-4*pi)
        morbLC = morbLC - (post_fermi+fermi)/2 * ch_t1t2.real/(-4*pi)
        morbIC = morbIC - (post_fermi+fermi)/2 * ch_t1t2.real/(-4*pi)
    elif gap_pos == "top":
        # morbTotal = morbTotal + 2 * post_fermi * ch_tau1tau2.real/(-4*pi)
        morbLC = morbLC - post_fermi * ch_t1t2.real/(-4*pi)
        morbIC = morbIC - post_fermi * ch_t1t2.real/(-4*pi)
    else:
        # morbTotal = morbTotal + 2 * fermi * ch_tau1tau2.real/(-4*pi)
        morbLC = morbLC - fermi * ch_t1t2.real/(-4*pi)
        morbIC = morbIC - fermi * ch_t1t2.real/(-4*pi)
        
    pre_processed = True

In [ ]:
# fermi = fermi[:,1::2]
# ch_tau1tau2 = ch_tau1tau2[:,1::2]
# morbTotal = morbTotal[:,1::2]
# morbLC = morbLC[:,1::2]
# morbIC = morbIC[:,1::2]

# fluxes = fluxes[1::2]

# Plots

In [ ]:
# Plot parameter
mpl.pyplot.rcdefaults()
plt.rcParams['figure.figsize'] = [12, 15]
plt.rcParams['savefig.facecolor'] = "white"
mpl.rcParams['figure.dpi'] = 300
mpl.rcParams['axes.linewidth'] = 1.2
mpl.rcParams['text.usetex'] = True
mpl.rcParams['mathtext.fontset'] = 'cm'
mpl.rcParams['font.family'] = 'cmu serif'
mpl.rcParams["axes.prop_cycle"] = plt.cycler( "color", cm.plasma(np.linspace(0,0.9,5)) )
# mpl.rcParams["axes.prop_cycle"] = cycler('color', ['#377eb8', '#ff7f00', '#4daf4a',
#                                                    '#f781bf', '#a65628', '#984ea3',
#                                                    '#999999', '#e41a1c', '#dede00'])

s=10
fs  = 12
lfs = 14
tfs = 14

pad = -10

In [ ]:
fig, ax = plt.subplots(4,figsize=(6,8))

for a in ax:
    a.set_xticks(np.linspace(0,0.5,6))
    a.xaxis.set_minor_locator(AutoMinorLocator())

# fig.suptitle(tex+r': t={:.0f}, m={:.0f}, size={:.0f}, shift={:.1f}, mag={:.1f} '.format(t,m,size,shift,mag)+'\n'+gap+' '+gap_pos, fontsize=16,y=0.99)

for i in range(0,n_flux,3):
    y_ticks = np.arange(-2, 2, 0.1)

    ax[0].plot(thetas,fermi[::,i].real/(4*pi),label=r'$\phi\approx{:.2f}$'.format(fluxes[i]));
    # ax[0].annotate(r'$\phi\approx{:.2f}$'.format(fluxes[i]),(thetas[-1]+0.01,fermi[i,-1]-0.2),fontsize=fs)
    # ax[0].set_yticks(y_ticks)
    ax[0].grid(True)
    ax[0].set_ylim([-0.3,0.5])
    # ax[0].set_xlabel(r'$\vartheta$',fontsize=lfs,labelpad=pad)
    ax[0].set_title(r'$\frac{E_F}{4\pi}\; / \; \lambda_\mathrm{hop}$',fontsize=tfs)

#     y_ticks = np.arange(-2, 2, 0.1)

#     ax[1].plot(thetas,ch_t1t2[::,i].real,label=r'$\phi\approx{:.2f}$'.format(fluxes[i]));
#     #ax[1].set_yticks(y_ticks)
#     ax[1].grid(True)
#     #ax[1].set_ylim([0, 0.5])
#     ax[1].set_xlabel(r'$\vartheta$',fontsize=lfs,labelpad=pad)
#     ax[1].set_title(r'$Ch_{t_1 t_2}$',fontsize=tfs)

    y_ticks = np.arange(-0.1, 0.06, 0.025)

    ax[1].plot(thetas,morbTotal[::,i].real,label=r'$\phi\approx{:.2f}$'.format(fluxes[i]));
    # ax[1].annotate(r'$\phi\approx{:.2f}$'.format(fluxes[i]),(thetas[-1]+0.01,morbTotal[i,-1]-0.01),fontsize=fs)
    ax[1].set_yticks(y_ticks)
    ax[1].grid(True)
    ax[1].set_ylim([-0.08, 0.055])
    # ax[1].set_xlabel(r'$\vartheta$',fontsize=lfs,labelpad=pad)
    ax[1].set_title(r'$M^z_\mathrm{orb}\; / \; \frac{e}{\hbar}\lambda_\mathrm{hop}$',fontsize=tfs)

    y_ticks = np.arange(-4, 4, 0.5)

    ax[2].plot(thetas,morbLC[::,i].real,label=r'$\phi\approx{:.2f}$'.format(fluxes[i]));
    # ax[2].annotate(r'$\phi\approx{:.2f}$'.format(fluxes[i]),(thetas[-1]+0.01,morbLC[i,-1]-0.01),fontsize=fs)
    # ax[2].set_yticks(y_ticks)
    ax[2].grid(True)
    ax[2].set_ylim([-0.17, 0.02])
    # ax[2].set_xlabel(r'$\vartheta$',fontsize=lfs,labelpad=pad)
    ax[2].set_title(r'$M^{LC}_\mathrm{orb}+\frac{E_F}{4\pi} Ch_{t_1 t_2}\; / \; \frac{e}{\hbar}\lambda_\mathrm{hop}$',fontsize=tfs)

    y_ticks = np.arange(-4, 4, 0.25)

    ax[3].plot(thetas,morbIC[::,i].real,label=r'$\phi\approx{:.2f}$'.format(fluxes[i]));
    # ax[3].annotate(r'$\phi\approx{:.2f}$'.format(fluxes[i]),(thetas[-1]+0.01,morbIC[i,-1]-0.001),fontsize=fs)
    # ax[3].set_yticks(y_ticks)
    ax[3].grid(True)
    ax[3].set_ylim([-0.02, 0.17])
    # ax[3].set_xlabel(r'$\vartheta$',fontsize=lfs,labelpad=pad)
    ax[3].set_title(r'$M^{IC}_\mathrm{orb}+\frac{E_F}{4\pi} Ch_{t_1 t_2}\; / \; \frac{e}{\hbar}\lambda_\mathrm{hop}$',fontsize=tfs)
    
for a in ax:
    a.set_xlim([-0.01,0.49])
    a.set_xlabel(r'$\vartheta$',fontsize=lfs,labelpad=pad)
    a.tick_params(axis='both', which='major', labelsize=fs)

ax[0].legend(loc=(0.554,0.02), ncol=2)

fig.tight_layout(h_pad=0.5, w_pad=0)
plt.savefig("./plots/morb_scaling_theta_U1.png", dpi=300, bbox_inches = 'tight')